***Creating the Eight Emotions Sets of Plutchik’s Wheel***

*I created the eight emotions sets of Plutchik’s Wheel using an English-English dictionary containing a large number of words with their explanations. Plutchik’s Wheel's primary emotions are joy, trust, fear, surprise, sadness, anticipation, anger, and disgust.*

*First, I built a dictionary, taking the words as 'key' and their numbers as 'value', and I included the words of the explanation of the first emotion with their numbers. After that, I expanded the dictionary by adding the words of the explanation of each 'key' in the dictionary. I can control the repetition of the process by the depth of the function I made. When the program encounters repeated words, it increases the 'value' according to their number.*

In [8]:
# Import the modules required
import json
import pandas as pd
from nltk.corpus import stopwords
import regex as re
from string import digits

In [78]:
# Reading the dictionary that will be used  to create the sets 
data = json.load(open(r"dictionary.json"))

In [79]:
# Set a function taking out related words for a given word and putting them all into a dictionary

def find_related(input_string: str, dictionary: dict, depth: int = 1):
    if input_string not in dictionary:
        return
# I used 'source and target' since we can't make changes in the dictionary in Python while traversing over it
    source = {input_string: 1}
    target = {}

    for i in range(depth):
        for word1 in source:
            if word1 not in dictionary:
                continue

            # Find the related words, remove symbols and digits, and split them to list
            description = dictionary[word1]
            res = ''.join([i for i in description if not i.isdigit()])
            words_list = re.sub(r'[^\w\s]', '', res.lower()).split()
            
            # Remove stop words with NLTK
            for word in words_list:
                if word in stopwords.words('english'):
                    words_list.remove(word)
                    continue

                # Build dictionary from list
                if word not in target:
                    target[word] = 1
                else:
                    target[word] += 1

        source.clear()
        source = target.copy()

    return target

In [80]:
# Run the function in order to create 'joy' dictionary in the depth of 4
joy = find_related('joy', data, depth=4)
print('The length of joy dictionary:', len(joy))

# Store the dictionary in a json file:
with open('sets\\joy.json', 'w') as fp:
    json.dump(joy, fp)

The length of joy dictionary: 44586


In [118]:
# Here is an example of what the 'joy' dictionary looks like
dict(list(joy.items())[:4])

{'excited': 90, 'acquisition': 33, 'pleasurable': 14, 'feelings': 113}

*Now, Let's do the same for the other 7 emotions*

In [81]:
trust = find_related('trust', data, depth=4)
print('The length of trust dictionary:', len(trust))
with open('sets\\trust.json', 'w') as fp:
    json.dump(trust, fp)
    
fear = find_related('fear', data, depth=4)
print('The length of fear dictionary:', len(fear))
with open('sets\\fear.json', 'w') as fp:
    json.dump(fear, fp)  
    
surprise = find_related('surprise', data, depth=4)
print('The length of surprise dictionary:', len(surprise))
with open('sets\\surprise.json', 'w') as fp:
    json.dump(surprise, fp)
    
sadness = find_related('sadness', data, depth=4)
print('The length of sadness dictionary:', len(sadness))
with open('sets\\sadness.json', 'w') as fp:
    json.dump(sadness, fp)
    
disgust = find_related('disgust', data, depth=4)
print('The length of disgust dictionary:', len(disgust))
with open('sets\\disgust.json', 'w') as fp:
    json.dump(disgust, fp)
    
anger = find_related('anger', data, depth=4)
print('The length of anger dictionary:', len(anger))
with open('sets\\anger.json', 'w') as fp:
    json.dump(anger, fp)
    
anticipation = find_related('anticipation', data, depth=4)
print('The length of anticipation dictionary:', len(anticipation))
with open('sets\\anticipation.json', 'w') as fp:
    json.dump(anticipation, fp)

The length of trust dictionary: 47199
The length of fear dictionary: 44471
The length of surprise dictionary: 43653
The length of sadness dictionary: 36553
The length of disgust dictionary: 41771
The length of anger dictionary: 45301
The length of anticipation dictionary: 40443


***Importing Tweets that be Analyzed Using Snscrape Libray***

In [41]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Creating list to append tweet data to
attributes_container = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('fifa world cup 2022 since:2022-12-01 until:2022-12-31').get_items()):
    if i > 100:
        break
    attributes_container.append([tweet.user.username, tweet.date, tweet.content])
    
# Creating a dataframe to load the list
tweets_df = pd.DataFrame(attributes_container, columns=["User", "Date Created", "Tweet"])

tweets = tweets_df['Tweet']

C:\Users\Abdalhadi\AppData\Local\Temp\ipykernel_4064\204851233.py:11: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  attributes_container.append([tweet.user.username, tweet.date, tweet.content])


***Set a Class for Applying Sentiment Analysis on the Tweets***

In [42]:
class Feeling:
    joy_dic = json.load(open(f"sets\\joy.json"))
    trust_dic = json.load(open(f"sets\\trust.json"))
    fear_dic = json.load(open(f"sets\\fear.json"))
    surprise_dic = json.load(open(f"sets\\surprise.json"))
    sadness_dic = json.load(open(f"sets\\sadness.json"))
    disgust_dic = json.load(open(f"sets\\disgust.json"))
    anger_dic = json.load(open(f"sets\\anger.json"))
    anticipation_dic = json.load(open(f"sets\\anticipation.json"))
    
    def __init__(self, tweet):
        self.joy = 0
        self.trust = 0
        self.fear = 0
        self.surprise = 0
        self.sadness = 0
        self.disgust = 0
        self.anger = 0
        self.anticipation = 0

        self.words = []
        self.split(tweet)
        self.find_frequency()

    def split(self, tweet):
        res = ''.join([i for i in tweet if not i.isdigit()])
        self.words = re.sub(r'[^\w\s]', '', res.lower()).split()
        for word in self.words:
            if word in stopwords.words('english'):
                self.words.remove(word)
                continue

    def find_frequency(self):
        #print("length of the text: ", len(self.words))
        for word in self.words:
            self.joy += Feeling.joy_dic.get(word, 0)
            self.trust += Feeling.trust_dic.get(word, 0)
            self.fear += Feeling.fear_dic.get(word, 0)
            self.surprise += Feeling.surprise_dic.get(word, 0)
            self.sadness += Feeling.sadness_dic.get(word, 0)
            self.disgust += Feeling.disgust_dic.get(word, 0)
            self.anger += Feeling.anger_dic.get(word, 0)
            self.anticipation += Feeling.anticipation_dic.get(word, 0)

***Run the Class and Arrange the Results in a Data Frame.***

In [43]:
# Create empty DataFrame with column names
df = pd.DataFrame(columns = ['Tweet', 'Joy', 'Trust', 'Fear', 'Surprise', 'Sadness', 'Disgust', 'Anger', 'Anticipation'])  

# Fill the DataFrame with the result of each tweet row by row
for twt in tweets:
    tweet = Feeling(twt)
    df.loc[len(df)] = [twt, tweet.joy, tweet.trust, tweet.fear, tweet.surprise,
                   tweet.sadness, tweet.disgust, tweet.anger, tweet.anticipation]

pd.options.display.max_colwidth = 280 
df.head()

,Tweet,Joy,Trust,Fear,Surprise,Sadness,Disgust,Anger,Anticipation
0,YOUTUBE • VÍDEOS EM ALTA GLOBAL 🌎\n\n#17 (-7) - 'It’s Beginning To Look A Lot Like Christmas' (cover) by V of BTS\n\n#35 (-21) - [BTS Episode] FIFA World Cup 2022 Soundtrack 'Dreamers' MV Shoot Sketch\n\n(@BTS_twt),3896,4554,3856,3588,2488,3217,3900,3043
1,"Fancy this Adidas FA Cymru 2022/23 Home Shirt from the 2022 FIFA World Cup costing 22 quid, well worth a bargain I'd say\n\n@adidas @adidasUK @adidasfootball @FAWales @FIFAWorldCup https://t.co/0bF7AntYyh",1780,2042,1740,1656,1242,1507,1804,1365
2,"Y’all keep forgetting “Messi + Fifa = Fifa World Cup 2022 “, he didn’t do it by himself.",743,859,733,707,542,634,736,590
3,RT bts_bighit: [Episode] #정국 #JungKook FIFA World Cup Qatar 2022 Soundtrack ‘Dreamers’ 뮤직비디오 촬영 현장 비하인드 공개! ⚽️🐰💜 #FIFAWorldCup #Dreamers20…,541,635,524,504,379,453,541,448
4,FIFA World Cup 2022: Fears grow over Argentina fan who went topless as new footage emerges from final against France https://t.co/GLm3kX6RBl final/news-story/533bb96cb29392b3a94bab517df57a37 via @newscoma Then FIFA should ban Arab countries from sponsoring the World Cup!,1847,2158,1827,1735,1221,1552,1860,1474
